# Solar Cell

## Photovoltaic Device Physics

Solar cells convert light energy into electrical energy using the photovoltaic effect. Understanding the device physics is essential for optimizing efficiency.

**Learning Objectives:**
- Understand solar cell operation
- Simulate dark I-V characteristics
- Analyze carrier generation and recombination
- Study surface recombination effects

---

## 1. Solar Cell Physics

### 1.1 Structure

```
    Light
      |
      v
  =========== Front Contact
  | N+ Emitter (thin)
  |-----------| Junction
  |           |
  | P Base    | (thick absorber)
  |           |
  =========== Back Contact
```

### 1.2 Key Parameters

- **Open-circuit voltage** ($V_{oc}$):
  $$V_{oc} = \frac{kT}{q}\ln\left(\frac{I_L}{I_0} + 1\right)$$

- **Short-circuit current** ($I_{sc}$): Equal to photogenerated current $I_L$

- **Fill Factor** (FF):
  $$FF = \frac{P_{max}}{V_{oc} \cdot I_{sc}}$$

- **Efficiency**:
  $$\eta = \frac{P_{max}}{P_{in}} = \frac{V_{oc} \cdot I_{sc} \cdot FF}{P_{in}}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nanohubpadre import create_solar_cell

# Physical constants
q = 1.6e-19
kT = 0.0259  # eV at 300K

print("Solar Cell Parameters:")
print("="*40)
print(f"Thermal voltage VT = {kT*1000:.1f} mV")
print(f"Silicon bandgap = 1.12 eV")
print(f"Typical Voc ~ 0.5-0.7 V for Si")

---

## 2. Creating a Solar Cell Simulation

In [ ]:
# Create an N-on-P solar cell
sim_solar = create_solar_cell(
    # Geometry
    emitter_depth=0.5,      # 500nm emitter
    base_thickness=200.0,   # 200um base (absorber)
    device_width=1.0,
    
    # Doping
    emitter_doping=1e19,    # N+ emitter
    base_doping=1e16,       # P base
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,               # SRH recombination
    auger=True,             # Auger recombination
    
    # Material parameters
    taun0=1e-5,             # Electron lifetime (10 us)
    taup0=1e-5,             # Hole lifetime (10 us)
    
    # Surface recombination
    front_surface_velocity=1e4,   # Front SRV (cm/s)
    back_surface_velocity=1e7,    # Back SRV (cm/s)
    
    # Output
    log_bands_eq=True
)

print("Solar Cell Configuration:")
print("="*40)
print("Structure: N-on-P")
print("Emitter: N+ 1e19 cm^-3, 0.5 um")
print("Base: P 1e16 cm^-3, 200 um")
print("Lifetimes: 10 us")

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_solar.generate_deck())

---

## 3. Dark I-V Characteristics

In [ ]:
# Dark I-V simulation
sim_dark = create_solar_cell(
    emitter_depth=0.5,
    base_thickness=200.0,
    emitter_doping=1e19,
    base_doping=1e16,
    taun0=1e-5,
    taup0=1e-5,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="dark_iv",
    
    # Forward bias sweep
    forward_sweep=(0.0, 0.7, 0.02)
)

print("Dark I-V Simulation")
print("="*40)
print("Voltage sweep: 0 to 0.7V")

In [ ]:
# Theoretical dark and illuminated I-V
V = np.linspace(0, 0.7, 100)
I0 = 1e-12  # Dark saturation current (A)
IL = 35e-3  # Photogenerated current (35 mA) - AM1.5

# Dark current
I_dark = I0 * (np.exp(V / kT) - 1)

# Illuminated current (superposition)
I_light = IL - I0 * (np.exp(V / kT) - 1)

plt.figure(figsize=(12, 5))

# Dark I-V
plt.subplot(1, 2, 1)
plt.semilogy(V, I_dark, 'b-', linewidth=2)
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (A)', fontsize=12)
plt.title('Dark I-V Characteristic', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.7)

# Illuminated I-V
plt.subplot(1, 2, 2)
plt.plot(V, I_light * 1e3, 'r-', linewidth=2)
plt.axhline(y=0, color='gray', linestyle='--')
plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (mA)', fontsize=12)
plt.title('Illuminated I-V (Theoretical)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.7)

# Mark key points
Voc = kT * np.log(IL/I0 + 1)
plt.axvline(x=Voc, color='g', linestyle=':', label=f'Voc = {Voc:.3f}V')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nTheoretical parameters:")
print(f"  Isc = {IL*1e3:.1f} mA")
print(f"  Voc = {Voc:.3f} V")

---

## 4. Effect of Carrier Lifetime

In [ ]:
# Compare different lifetimes
lifetimes = [1e-7, 1e-6, 1e-5, 1e-4]  # seconds

print("Lifetime Comparison:")
print("="*50)

for tau in lifetimes:
    # Diffusion length L = sqrt(D*tau)
    D = 35  # Diffusion coefficient (cm^2/s) for electrons in p-Si
    L = np.sqrt(D * tau) * 1e4  # Convert to um
    
    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        taun0=tau,
        taup0=tau,
        log_iv=True,
        forward_sweep=(0.0, 0.7, 0.02)
    )
    
    print(f"tau = {tau:.0e} s: L = {L:.1f} um")

In [ ]:
# Visualize effect of diffusion length
D = 35  # cm^2/s
tau_values = np.logspace(-7, -3, 50)
L_values = np.sqrt(D * tau_values) * 1e4  # um

# Base thickness
W_base = 200  # um

# Collection efficiency (simplified)
# eta ~ L/(L+Lb) for optically thin
eta_collection = L_values / (L_values + W_base/2)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.loglog(tau_values * 1e6, L_values, 'b-', linewidth=2)
plt.axhline(y=W_base, color='r', linestyle='--', label=f'Base thickness = {W_base} um')
plt.xlabel('Lifetime (us)', fontsize=12)
plt.ylabel('Diffusion Length (um)', fontsize=12)
plt.title('Diffusion Length vs Lifetime', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.semilogx(tau_values * 1e6, eta_collection * 100, 'g-', linewidth=2)
plt.xlabel('Lifetime (us)', fontsize=12)
plt.ylabel('Collection Efficiency (%)', fontsize=12)
plt.title('Collection Efficiency vs Lifetime', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0, 100)

plt.tight_layout()
plt.show()

print("\nKey insight: L > W_base for good collection")

---

## 5. Surface Recombination Effects

In [ ]:
# Compare surface recombination velocities
srv_values = [1e2, 1e4, 1e6]  # cm/s

print("Surface Recombination Comparison:")
print("="*50)

for srv in srv_values:
    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        front_surface_velocity=srv,
        back_surface_velocity=srv,
        log_iv=True,
        forward_sweep=(0.0, 0.7, 0.02)
    )
    
    print(f"SRV = {srv:.0e} cm/s")

print("\nLower SRV -> Higher Voc and efficiency")

---

## 6. Complete Simulation Example

In [ ]:
# Complete solar cell characterization
sim_complete = create_solar_cell(
    # Geometry
    emitter_depth=0.5,
    base_thickness=200.0,
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny=150,
    
    # Doping
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    conmob=True,
    fldmob=True,
    
    # Material
    taun0=1e-5,
    taup0=1e-5,
    
    # Surface recombination
    front_surface_velocity=1e3,  # Well-passivated front
    back_surface_velocity=1e5,   # BSF
    
    # Output
    log_iv=True,
    iv_file="solar_complete",
    log_bands_eq=True,
    
    # Dark I-V sweep
    forward_sweep=(0.0, 0.75, 0.01)
)

print("Complete Solar Cell Simulation")
print("="*50)
print(sim_complete.generate_deck())

---

## Summary

In this notebook, you learned:

1. **Solar Cell Structure**: N-on-P or P-on-N junction
2. **Dark I-V**: Diode characteristic (exponential)
3. **Key Parameters**: Voc, Isc, FF, efficiency
4. **Lifetime Effects**: Longer lifetime -> better collection
5. **Surface Recombination**: Critical for efficiency

**Key Equations:**
- $V_{oc} = \frac{kT}{q}\ln(I_L/I_0 + 1)$
- $L = \sqrt{D\tau}$ (diffusion length)

**Next**: [07 - MOS Capacitor](07_MOS_Capacitor.ipynb) - Capacitance-voltage analysis